    Таблица visits (лог сервера с информацией о посещениях сайта):

    User Id — уникальный идентификатор пользователя
    Device — категория устройства пользователя
    Session start — дата и время начала сессии
    Session End — дата и время окончания сессии
    Channel — идентификатор рекламного источника, из которого пришел пользователь
    Region - страна пользователя
    
    Таблица orders (информация о заказах):

    User Id — уникальный id пользователя, который сделал заказ
    Event Dt — дата и время покупки
    Revenue — выручка
    
    Таблица costs (информация о затратах на маркетинг):

    Channel — идентификатор рекламного источника
    Dt — дата
    Costs — затраты на этот рекламный источник в этот день

    LTV -  общая сумма денег, которую один клиент в среднем приносит компании (выручка еа текущий день / размер когорты) (1, 3, 7, 14 дней)
    ROI - окупаемость LTV/САС
    САС - стоимость привлечения одного клиента ( Расход на рекламу / размер когорты )
    ARPU - средняя выручка с пользователя 
    ARPPU - средняя выручка с платящего пользователя
    удержание 
    CTR - показов рекламного баннера в клики называют(конверсия клики/показы)
    CR - переход пользователя от одного состояния в другое(конверсия)
    DAU - количество уникальных пользователей за день 
    WAU - количество уникальных пользователей за неделю 
    MAU - количество уникальных пользователей за месяц 
    ASL - Средняя продолжительность сессии 
    CPM -  стоимость тысячи показов рекламного объявления.
    RPM - доход за тысячу показов.
    СPC - оплата за каждого пользователя, кликнувшего на рекламное объявление, то есть перешедшего на сайт.
    CPL - оплата за каждого пользователя, оставившего свои контакты
    CPA - оплата за определённое действие, совершённое пользователем. Например, за покупку.

Загружаем библиотеки

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

Загружаем таблицы

In [2]:
job = 'C:\\Users\\a.vechkanov\\Desktop\\Госты, Литература\\Остальное\\Data analitics\\Проекты\\Продуктовые метрики\\'
home = 'C:\\Users\\Александр\\OneDrive\\Рабочий стол\\Python\\ЯндексПрактикум\\Project\\Продуктовые метрики\\'
try:   
    visits = pd.read_csv(
        job+'visits_info_short.csv'
    )
    
    orders = pd.read_csv(
        job+'orders_info_short.csv'
    )
        
    costs = pd.read_csv(
        job+'costs_info_short.csv'
    )
    
except:
    visits = pd.read_csv(
        home+'visits_info_short.csv'
    )
    orders = pd.read_csv(
        home+'orders_info_short.csv'
    )
    costs = pd.read_csv(
        home+'costs_info_short.csv'
    )
    

Подготовим наши данные к работе

In [3]:
names = ['visits', 'orders', 'costs']
tables = [visits, orders, costs]
for table,name in zip(tables, names):
    print('ТАБЛИЦА: ',name)
    display(table.shape)
    display(table.sample(3))
    table.info()
    print('\n')

ТАБЛИЦА:  visits


(309901, 6)

,User Id,Region,Device,Channel,Session Start,Session End
251250,172005068825,Germany,PC,OppleCreativeMedia,2019-07-28 18:20:12,2019-07-28 18:36:16
122283,121675088294,United States,iPhone,TipTop,2019-08-26 15:57:46,2019-08-26 17:38:20
15544,136522509076,United States,Mac,TipTop,2019-05-17 23:39:04,2019-05-18 00:30:38


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   User Id        309901 non-null  int64 
 1   Region         309901 non-null  object
 2   Device         309901 non-null  object
 3   Channel        309901 non-null  object
 4   Session Start  309901 non-null  object
 5   Session End    309901 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.2+ MB


ТАБЛИЦА:  orders


(40212, 3)

,User Id,Event Dt,Revenue
4761,535839189397,2019-06-25 06:01:16,4.99
11499,219335074792,2019-08-08 10:45:43,4.99
26388,620496139495,2019-10-24 18:57:37,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   User Id   40212 non-null  int64  
 1   Event Dt  40212 non-null  object 
 2   Revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


ТАБЛИЦА:  costs


(1800, 3)

,dt,Channel,costs
270,2019-07-30,MediaTornado,3.84
168,2019-10-16,FaceBoom,154.00
160,2019-10-08,FaceBoom,201.30


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   Channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB




Пропусков нет, проверим данные на дубликаты

In [4]:
for table in tables:
    print(table.duplicated().sum())

0
0
0


Приведем таблицы к удобному для работы виду 

In [5]:
visits.columns

Index(['User Id', 'Region', 'Device', 'Channel', 'Session Start',
       'Session End'],
      dtype='object')

In [6]:
visits = visits.rename(
    columns = {
        visits.columns[0]:'user_id',
        visits.columns[1]:'region',
        visits.columns[2]:'device',
        visits.columns[3]:'channel',
        visits.columns[4]:'session_start',
        visits.columns[5]:'session_end'
    }
)

orders = orders.rename(
    columns = {
        orders.columns[0]:'user_id',
        orders.columns[1]:'event_dt',
        orders.columns[2]:'revenue',
    }
)

costs = costs.rename(
    columns = {
        costs.columns[0]:'dt',
        costs.columns[1]:'channel',
        costs.columns[2]:'costs',
    }
)

Посчитаем длину сессии пльзователей из таблицы visits

In [7]:
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])

Преобразуем тип данных

In [8]:
costs['dt'] = pd.to_datetime(costs['dt'])

In [9]:
visits['session_length'] = pd.to_datetime(visits['session_end']) - pd.to_datetime(visits['session_start'])

visits.query('user_id == 267068500935')

,user_id,region,device,channel,session_start,session_end,session_length
110558,267068500935,United States,iPhone,organic,2019-08-16 02:08:57,2019-08-16 02:16:38,0 days 00:07:41
111588,267068500935,United States,iPhone,organic,2019-08-17 13:27:23,2019-08-17 14:15:08,0 days 00:47:45
121264,267068500935,United States,iPhone,organic,2019-08-25 18:45:22,2019-08-25 18:47:31,0 days 00:02:09


Создадим функцию для создания профилелй пользователей

In [11]:
# выделяем первое песещение пользователя 
profiles = visits.pivot_table(
    index = 'user_id',
    values = ['session_start', 'channel', 'device', 'region'],
    aggfunc = 'first'
). reset_index().sort_values(by = 'session_start')

# выделяем месяц
profiles['month'] = profiles['session_start'].astype('datetime64[M]')
profiles['dt'] = profiles['session_start'].astype('datetime64[D]')

# является ли платящим(хоть раз платил) клиент или нет(пересечение в таблице orders)
profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique()) 

# посчитаем стоимость привлечения одного пользователя
acquisition_cost = profiles.pivot_table(index = ['dt','channel'], values = 'user_id', aggfunc = 'nunique').reset_index().merge(
        costs.pivot_table(index = ['dt','channel'], values = 'costs').reset_index(), 
        on = ['dt','channel'],
        how = 'left'
                      ).rename(columns = {'user_id':'count_user_id'})

acquisition_cost['acquisition_cost'] = acquisition_cost['costs']/acquisition_cost['count_user_id']

profiles = profiles.merge(acquisition_cost, on=['dt','channel'], how='left')

profiles.head()

,user_id,channel,device,region,session_start,month,dt,payer,count_user_id,costs,acquisition_cost
0,382394863972,MediaTornado,Mac,United States,2019-05-01 00:00:41,2019-05-01,2019-05-01,False,26,6.24,0.24
1,428935481941,AdNonSense,PC,UK,2019-05-01 00:01:35,2019-05-01,2019-05-01,False,39,40.95,1.05
2,86266511938,organic,Android,Germany,2019-05-01 00:02:38,2019-05-01,2019-05-01,False,519,NaN,NaN
3,763000657462,organic,PC,UK,2019-05-01 00:04:21,2019-05-01,2019-05-01,False,519,NaN,NaN
4,791058813280,LeapBob,Android,France,2019-05-01 00:05:17,2019-05-01,2019-05-01,False,12,2.52,0.21


In [ ]:


def get_profiles(sessions, orders, ad_costs): #events, 3 поз. и , event_names=[] 5 поз.

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date.astype('datetime64') # ПОМЕНЯЛ ТИП ДАННЫХ 
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # добавляем флаги для всех событий из event_names
#     for event in event_names:
#         if event in events['event_name'].unique():
#             profiles[event] = profiles['user_id'].isin(
#                 events.query('event_name == @event')['user_id'].unique()
#             )

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles